In [158]:
import random
from math import ceil,log10
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *

In [12]:
def generatePairs(n_examples, n_numbers, largest):
  '''
  Parameters
  ----------
  n_examples : int
      The number of examples to generate
  n_numbers : int
      The number of integers to be included per example
  largest : int
      The largest random number allowed to generate

  Returns
  -------
  x and y list
      2 lists corresponding to feature and label
  '''
  
  x, y = [], []
  #Generate n_examples number of lists 
  for i in range(n_examples):
    #Get a list of random numbers
    input = [random.randint(1,largest) for _ in range(n_numbers)]
    #Get the sum of all the numbers in the list
    output = sum(input)
    x.append(input)
    y.append(output)

  return x, y

In [41]:
def to_string(X, y, n_numbers, largest):
  max_length = n_numbers * ceil(log10(largest+1)) + n_numbers - 1
  Xstr = []
  for pattern in X:
    strp = '+' .join([str(n) for n in pattern])
    strp = ' '*(max_length-len(strp)) + strp
    Xstr.append(strp)
  max_length = ceil(log10(n_numbers * (largest+1)))
  ystr = []
  for pattern in y:
    strp = str(pattern)
    strp = ' '*(max_length-len(strp)) + strp
    ystr.append(strp)
  return Xstr, ystr

In [103]:
def encode(x,y):
  alphabet = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '+', ' ']
  xenc = []
  yenc = []
  char2idx = dict((char,idx) for idx,char in enumerate(alphabet))
  #Loop through each sample in x (x contains multiple samples)
  for sample in x:
    #Loop through each character in the sample
    xenc.append([char2idx[char] for char in sample])
  for sample in y:
    yenc.append([char2idx[char] for char in sample])
  
  return xenc, yenc

In [113]:
def ohe(x,y, depth=len(alphabet)):
  xenc = []
  for sample in x:
    sampleEnc = []
    for char in sample:
      vect = [0 for _ in range(depth)]
      vect[char] = 1
      sampleEnc.append(vect)
    xenc.append(sampleEnc)

  yenc = []
  for sample in y:
    sampleEnc = []
    for char in sample:
      vect = [0 for _ in range(depth)]
      vect[char] = 1
      sampleEnc.append(vect)
    yenc.append(sampleEnc)

  return xenc, yenc

In [114]:
xenc, yenc = encode(x,y)

In [115]:
print(xenc)

[[11, 11, 1, 0, 10, 9, 10, 2], [11, 11, 11, 4, 10, 6, 10, 4]]


In [117]:
xohe, yohe = ohe(xenc, yenc)

In [120]:
yohe

[[[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]]]

In [164]:
def generate_seq(n_examples, n_numbers, largest):
  x,y = generatePairs(n_examples, n_numbers, largest)
  x,y = to_string(x,y, n_numbers, largest)
  xenc, yenc = encode(x, y)
  xohe, yohe = ohe(xenc, yenc)

  xohe = np.array(xohe)
  yohe = np.array(yohe)

  return xohe, yohe

In [147]:
def decode(seq, alphabet = alphabet):
  idx2char = dict((i,j) for i,j in enumerate(alphabet))
  decoded = []
  for ohechar in seq:
    decoded.append(idx2char[np.argmax(ohechar)])
  
  return ''.join(decoded)

In [153]:
xohe, yohe = generate_seq(n_samples, n_numbers, largest)

In [155]:
#Number of numbers per example
n_numbers = 3
#largest possible number
largest = 10

In [156]:
#Set of possible values any character can take
n_chars = len(alphabet)
#Maximum possible length for the input (x). All x are padded to this length
in_seq = n_numbers * ceil(log10(largest+1)) + n_numbers - 1
#Maximum possible length for the output (y). All y are padded to this length
out_seq = ceil(log10(n_numbers * (largest+1)))

In [160]:
model = Sequential([
                    #Define the Encoder
                    #in_seq is the number of time steps of the input seq (each timestep is one character ( 0-9 + ' ') )
                    LSTM(75, input_shape=(in_seq, n_chars)),
                    #Convert the 2D output of the encoder LSTM to a 3D output that can then be fed into the Decoder LSTM
                    #Repeats the vector for the number of characters in the output (2 in this case since 30 is the maximum possible value for y)
                    RepeatVector(out_seq),

                    #Define the Decoder
                    LSTM(50, return_sequences=True),
                    #Since you return sequences, you want to apply a Dense layer to each output sequence and use timedistributed to do so
                    TimeDistributed(Dense(n_chars, activation='softmax'))
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 75)                26400     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 2, 75)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 2, 50)             25200     
_________________________________________________________________
time_distributed (TimeDistri (None, 2, 12)             612       
Total params: 52,212
Trainable params: 52,212
Non-trainable params: 0
_________________________________________________________________


In [166]:
x,y = generate_seq(100000, n_numbers,largest)
model.fit(x,y, epochs=1, batch_size=32)

3125/3125 [==============================] - 23s 7ms/step - loss: 0.0896 - acc: 0.9858


In [167]:
x_test, y_test = generate_seq(100, n_numbers, largest)
model.evaluate(x_test, y_test)

4/4 [==============================] - 0s 3ms/step - loss: 0.0271 - acc: 0.9950


[0.02708352729678154, 0.9950000047683716]